In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("../../datasets/dataset_removed_repeated_charcter.csv")
df.head()

,comment_text,span,label
0,Great post !\r\nJustin at the 150 apologizing...,[],0
1,Mark Hume has always been the stereotypical Va...,"[149, 150, 151, 152, 153, 154, 155, 156, 157, ...",1
2,"Freakin maggots, I swear.",[],1
3,so far the stupidest comments are about the NR...,[],1
4,‘Stupidity is a more dangerous enemy of the go...,[],0


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X=df["comment_text"]
Y=df["label"]
print(X)
print(Y)

0         Great post !\r\nJustin at the 150 apologizing...
1        Mark Hume has always been the stereotypical Va...
2                                Freakin maggots, I swear.
3        so far the stupidest comments are about the NR...
4        ‘Stupidity is a more dangerous enemy of the go...
                               ...                        
20466    He might have been sleazy but trump is both sl...
20467    Seems like the single payer system would be be...
20468                                               Idiot!
20469    My only Question is:  3 staff attorneys chosen...
20470    "They look like fools", ignorance must be blis...
Name: comment_text, Length: 20471, dtype: object
0        0
1        1
2        1
3        1
4        0
        ..
20466    0
20467    0
20468    1
20469    0
20470    1
Name: label, Length: 20471, dtype: int64


In [5]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [6]:
from transformers import BertTokenizer,BertModel

c:\users\ravic\appdata\local\programs\python\python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
MAX_LEN=512

In [8]:
output=tokenizer.encode_plus("Hello all",max_length=MAX_LEN,padding='max_length',
                return_token_type_ids=True,
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt')
output
output['input_ids'].flatten().shape

torch.Size([512])

In [34]:
import torch
import torch.nn as nn

class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, df, tokenizer, max_len):
            self.df = df
            self.tokenizer = tokenizer
            self.max_len= max_len
            self.title=X
            self.targets=Y
        def __len__(self):
            return len(self.title)

        def __getitem__(self,index):
            title = str(self.title[index])
            title = " ".join(title.split())

            inputs = self.tokenizer.encode_plus(
                title,None,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                return_token_type_ids=True,
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )

            # print(f"[+]Index ok {index}",inputs['input_ids'].flatten().shape)
            # print(f"[^]Have text {title}")
            # return {
            #     'input_ids': inputs['input_ids'].flatten(),
            #     'attention_mask': inputs['attention_mask'].flatten(),
            #     'token_type_ids': inputs['token_type_ids'].flatten(),
            #     'targets': torch.FloatTensor(self.targets[index])
            # }
            target=self.targets[index]
            return inputs,target
                

In [35]:
train_size=0.8
train_df={}
train_df["message"]=X_train
train_df["label"]=Y_train

train_df=pd.DataFrame(train_df)


val_df={}

val_df["message"]=X_test
val_df["label"]=Y_test
val_df=pd.DataFrame(val_df)

print(train_df.head())
print(val_df.head())

                                                 message  label
17242                                         Good work.      0
5905   " This will change his life in very difficult ...      1
6803   Understatement of the year.  The Browns should...      1
3417   Fox and Friends features some of the most clue...      1
8554   Perhaps the point being missed is that the cur...      0
                                                 message  label
6688   Hey, chinie boy, TAKE A FLYING  F--K  AT\r\nA ...      0
13630  The only thing that outrages me is the Conserv...      0
8630   There is dumb and then there is Maxine-Pad Wat...      1
10787  yes the sky is still falling regardless of the...      1
18033  Roughly $1820 per month for a $hit-hole.  INSA...      0


In [36]:
train_dataset=CustomDataset(train_df,tokenizer,MAX_LEN)
val_dataset=CustomDataset(val_df,tokenizer,MAX_LEN)


In [37]:
train_data_loader=torch.utils.data.DataLoader(
    train_dataset,
    # shuffle=True,
    batch_size=64,
    num_workers=0

)
val_data_loader=torch.utils.data.DataLoader(
    val_dataset,
    # shuffle=True,
    batch_size=64,
    num_workers=0

)

In [38]:
device= torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [39]:
class ToxicClassiferModel(nn.Module):

    def __init__(self):
        super().__init__()

        self.BERTLayer=BertModel.from_pretrained("bert-base-uncased",return_dict=True)
        self.dropoutLayer=nn.Dropout(0.3)
        self.fc=nn.Linear(768,1)
    
    def forward(self,input_ids,attention_mask,token_type_id):
        output=self.BERTLayer(input_ids,attention_mask,token_type_id)
        output=self.dropoutLayer(output)
        output=self.fc(output)

        return output

model=ToxicClassiferModel()
model.to(device)


ToxicClassiferModel(
  (BERTLayer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [40]:
def loss_fn(outputs,target):
    return nn.BCEWithLogitsLoss()(outputs,target)

optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001)

In [ ]:
epochs=10

for epoch in range(epochs):
    val_loss=0
    model.train()
    try:
        for index,(inputs,traget) in enumerate(train_data_loader):
            print(index ,inputs['input_ids'])
    except Exception as e:
        print(f'[+]Exception catch {e}')

0 tensor([[[  101,  2307,  2695,  ...,     0,     0,     0]],

        [[  101,  2928, 20368,  ...,     0,     0,     0]],

        [[  101, 11576,  2378,  ...,     0,     0,     0]],

        ...,

        [[  101,  2347,  1005,  ...,     0,     0,     0]],

        [[  101, 14649,  1010,  ...,     0,     0,     0]],

        [[  101,  2122, 15967,  ...,     0,     0,     0]]])
1 tensor([[[  101, 16091,  1996,  ...,     0,     0,     0]],

        [[  101,  1045,  2123,  ...,     0,     0,     0]],

        [[  101, 21591, 16939,  ...,     0,     0,     0]],

        ...,

        [[  101,  2043,  2097,  ...,     0,     0,     0]],

        [[  101,  1047, 25399,  ...,     0,     0,     0]],

        [[  101,  2323,  1005,  ...,     0,     0,     0]]])
2 tensor([[[  101,  1045,  2064,  ...,     0,     0,     0]],

        [[  101,  9805,  2361,  ...,     0,     0,     0]],

        [[  101,  1996,  4365,  ...,     0,     0,     0]],

        ...,

        [[  101,  2023,  2003,  ..., 